# Holotomography to Minecraft

**Author**: Kevin Tan\
**Date**: 10/14/2024\
**Description**: A step-by-step guide to converting a Holotomography image taken on the Nanolive 3D Cell Explorer microscope to a Minecraft schematic. The resulting schematic can be loaded into Minecraft with Amulet or Worldedit.\
**Python Version**: Python 3.9.19 (recommended)

## Part 1 - Segmentation with ImageJ

### PyImageJ installation requires conda (or mamba)
You can install Miniconda [here](https://docs.anaconda.com/miniconda/)

### Install PyImageJ
Activate conda-forge
```
conda config --add channels conda-forge
conda config --set channel_priority strict
```

Install PyImageJ with OpenJDK 11
```
conda install pyimagej openjdk=11
```

More information can be found here:
https://py.imagej.net/en/latest/Install.html#installation

### Initialize PyImageJ

In [1]:
import imagej

ij = imagej.init(['sc.fiji:fiji:2.14.0', 'fr.inra.ijpb:MorphoLibJ_:1.6.2', 'org.framagit.mcib3d:mcib3d-plugins:4.0.63'], mode='interactive')

### Specify directories

In [2]:
from pathlib import Path
input_path = "raw_holotomography/breast_cancer.tiff"
#input_path = "raw_holotomography/breast_normal.tiff"
output_dir = "output"
Path(output_dir).mkdir(exist_ok=True)
title = Path(input_path).stem

### Open a holotomographic image

In [3]:
orig = ij.io().open(input_path)
ij.ui().show(orig)

### Crop, interpolate and filter the image

In [4]:
processed_macro = """
#@ String title

selectImage(title + ".tiff");
title = title + "_processed.tiff";
run("Duplicate...", "title=[title] duplicate");
run("Properties...", "channels=1 slices=96 frames=1 pixel_width=1.0000 pixel_height=1.0000 voxel_depth=1.0000");
run("Specify...", "width=440 height=440 x=36 y=36 slice=1");
run("Crop");
//run("Size...", "width=440 height=440 depth=176 interpolation=Bilinear");
run("Enhance Contrast...", "saturated=0.35 process_all use");
setOption("ScaleConversions", true);
run("8-bit");
run("Subtract Background...", "rolling=50 stack");
"""

result = ij.py.run_macro(processed_macro, {'title': title})

### Automatically segment the cytoplasm

In [5]:
segment_cyto_macro = """
#@ String title

title_processed = title + "_processed.tiff";
selectImage(title_processed);
temp = title + "_temp";
arg = "title=" + temp + " duplicate";
run("Duplicate...", arg);
run("Median...", "radius=5 stack");
setAutoThreshold("Li dark stack");
//run("Threshold...");
run("Make Binary", "black");
run("Morphological Filters (3D)", "operation=Closing element=Ball x-radius=7 y-radius=7 z-radius=3");
close(temp);
title = title + "_cyto.tiff";
rename(title);
"""

result = ij.py.run_macro(segment_cyto_macro, {'title': title})

### Automatically segment the cell membrane

In [6]:
segment_membrane_macro = """
#@ String title

title_cyto = title + "_cyto.tiff";
title_erosion = title + "_cyto-Erosion";
title_membrane = title + "_membrane.tiff";
selectImage(title_cyto);
run("Morphological Filters (3D)", "operation=Erosion element=Ball x-radius=1 y-radius=1 z-radius=1");
imageCalculator("Subtract create stack", title_cyto, title_erosion);
rename(title_membrane);
close(title_erosion);
"""

result = ij.py.run_macro(segment_membrane_macro, {'title': title})

### Manually segment the nuclei
This will open up a GUI to help with segmentation\
Read the instructions at https://imagej.net/plugins/segmentation-editor#labelling

In [7]:
segment_nuclei_macro = """
#@ String title

title_processed = title + "_processed.tiff";
title_nuclei = title + "_nuclei.tiff";
selectImage(title_processed);
call("Segmentation_Editor.newSegmentationEditor");
// Reset the material list
call("Segmentation_Editor.newMaterials");
call("Segmentation_Editor.addMaterial", "Nuclei", 255, 255, 255);
"""

result = ij.py.run_macro(segment_nuclei_macro, {'title': title})

When finished with segmentation click Ok and run the following code:

In [8]:
segment_nuclei_step2_macro = """
#@ String title

title_cyto = title + "_cyto.tiff";
title_nuclei = title + "_nuclei.tiff";
title_labels = title + "_processed.labels";
selectImage(title_labels);
arg = "title=" + title_nuclei + " duplicate";
run("Duplicate...", arg);
setThreshold(1, 65535, "raw");
run("Make Binary", "black");
imageCalculator("AND stack", title_nuclei, title_cyto);
run("Grays");
selectImage(title_labels);
run("Select All");
setBackgroundColor(0, 0, 0);
run("Clear", "stack");
run("Select None");
"""

result = ij.py.run_macro(segment_nuclei_step2_macro, {'title': title})

OPTIONAL - Smooth the nuclei so they are more round

In [9]:
smooth_nuclei_macro = """
#@ String title

title_nuclei = title + "_nuclei.tiff";
selectImage(title_nuclei);
run("Gaussian Blur 3D...", "x=3 y=3 z=3");
setThreshold(128, 255, "raw");
run("Make Binary", "background=Dark black");
"""

result = ij.py.run_macro(smooth_nuclei_macro, {'title': title})

### Automatically segment the nuclear membrane

In [10]:
segment_nuclei_membrane_macro = """
#@ String title

title_nuclei = title + "_nuclei.tiff";
title_erosion = title + "_nuclei-Erosion";
title_nuclei_membrane = title + "_nuclei_membrane.tiff";
selectImage(title_nuclei);
run("Morphological Filters (3D)", "operation=Erosion element=Ball x-radius=1 y-radius=1 z-radius=1");
imageCalculator("Subtract create stack", title_nuclei, title_erosion);
rename(title_nuclei_membrane);
close(title_erosion);
"""

result = ij.py.run_macro(segment_nuclei_membrane_macro, {'title': title})

### Manually segment the nucleoli

In [11]:
segment_nucleoli_macro = """
#@ String title

title_processed = title + "_processed.tiff";
title_nucleoli = title + "_nucleoli.tiff";
selectImage(title_processed);
call("Segmentation_Editor.newSegmentationEditor");
// Reset the material list
call("Segmentation_Editor.newMaterials");
call("Segmentation_Editor.addMaterial", "Nucleoli", 255, 255, 255);
"""

result = ij.py.run_macro(segment_nucleoli_macro, {'title': title})

When finished with segmentation click Ok and run the following code:

In [12]:
segment_nucleoli_step2_macro = """
#@ String title

title_nuclei = title + "_nuclei.tiff";
title_nucleoli = title + "_nucleoli.tiff";
title_labels = title + "_processed.labels";
selectImage(title_labels);
arg = "title=" + title_nucleoli + " duplicate";
run("Duplicate...", arg);
setThreshold(1, 65535, "raw");
run("Make Binary", "black");
imageCalculator("AND stack", title_nucleoli, title_nuclei);
run("Grays");
selectImage(title_labels);
run("Select All");
setBackgroundColor(0, 0, 0);
run("Clear", "stack");
run("Select None");
"""

result = ij.py.run_macro(segment_nucleoli_step2_macro, {'title': title})

OPTIONAL - Smooth the nucleoli so they are more round

In [13]:
smooth_nucleoli_macro = """
#@ String title

title_nucleoli = title + "_nucleoli.tiff";
selectImage(title_nucleoli);
run("Gaussian Blur 3D...", "x=3 y=3 z=3");
setThreshold(128, 255, "raw");
run("Make Binary", "background=Dark black");
"""

result = ij.py.run_macro(smooth_nucleoli_macro, {'title': title})

### Automatically segment organelles

In [14]:

segment_organelles_macro = """
#@ String title

title_processed = title + "_processed.tiff";
title_organelle = title + "_organelle.tiff";
title_cyto = title + "_cyto.tiff";
title_nuclei = title + "_nuclei.tiff";
temp = title + "_temp";
selectImage(title_processed);
run("3D Iterative Thresholding", "min_vol_pix=600 max_vol_pix=12000 min_threshold=100 min_contrast=5 criteria_method=VOLUME threshold_method=STEP segment_results=All value_method=5");
rename(temp);
setThreshold(1, 65535, "raw");
run("Make Binary", "black");
run("Arrange Channels...", "new=1");
run("Morphological Filters (3D)", "operation=Dilation element=Ball x-radius=1 y-radius=1 z-radius=1");
rename(title_organelle);
close(temp);
imageCalculator("AND stack", title_organelle, title_cyto);
imageCalculator("Subtract stack", title_organelle, title_nuclei);
"""

result = ij.py.run_macro(segment_organelles_macro, {'title': title})

### Automatically segment the organelle membranes

In [15]:
segment_organelle_membrane_macro = """
#@ String title

title_organelle_membrane = title + "_organelle_membrane.tiff";
title_erosion = title + "_organelle-Erosion";
title_organelle = title + "_organelle.tiff";
selectImage(title_organelle);
run("Morphological Filters (3D)", "operation=Erosion element=Ball x-radius=1 y-radius=1 z-radius=1");
imageCalculator("Subtract create stack", title_organelle, title_erosion);
rename(title_organelle_membrane);
close(title_erosion);
"""

result = ij.py.run_macro(segment_organelle_membrane_macro, {'title': title})

### Save all open images
Close any images that you don't want to be saved

In [16]:
save_macro = r"""
#@ String output_dir
output_dir = output_dir + "\\";

for (i=0; i<nImages; i++) {
        selectImage(i+1);
        title = getTitle;
        print(output_dir + title);
        saveAs("tiff", output_dir + title);
} 
"""

result = ij.py.run_macro(save_macro, {'output_dir': output_dir})

OPTIONAL - save a copy of the tiff files as npy

In [31]:
import tifffile
import numpy as np
for tiff_path in Path(output_dir).glob("*.tiff"):
    img = tifffile.imread(tiff_path)
    img = np.moveaxis(img, 0, 1) # rearrange axes to be minecraft (x, y, z)
    bool_img = img.astype("bool")
    np.save(tiff_path.with_suffix(".npy"), bool_img)

### Simple quantitative analysis

In [24]:
import tifffile

cyto = tifffile.imread(output_dir + "\\" + title + "_cyto.tiff")
nuclei = tifffile.imread(output_dir + "\\" + title + "_nuclei.tiff")
nucleoli = tifffile.imread(output_dir + "\\" + title + "_nucleoli.tiff")

print(f"{title} cyto sum: ", cyto.sum())
print(f"{title} nuclei sum: ", nuclei.sum())
print(f"{title} nucleoli sum: ", nucleoli.sum())
nc_ratio = nuclei.sum() / cyto.sum()
print(f"{title} nuclei cytoplasm ratio: {nc_ratio}")
nn_ratio = nucleoli.sum() / nuclei.sum()
print(f"{title} nucleoli nuclei ratio: {nn_ratio}")

breast_cancer cyto sum:  723402615
breast_cancer nuclei sum:  64263315
breast_cancer nucleoli sum:  3317040
breast_cancer nuclei cytoplasm ratio: 0.08883478393287257
breast_cancer nucleoli nuclei ratio: 0.05161638486903453


# Part 2 - Create a Schematic for import into Minecraft

The following code uses `mcschematic` to create a `.schem` file that can be imported into Minecraft using Amulet or Worldedit. For a more interactive experience, the GUI can be used for this step.

### Install mcschematic with pip

```
pip install mcschematic
```

In [25]:
import mcschematic
import tifffile
import numpy as np
from pathlib import Path

def write_image(schem : mcschematic.MCSchematic, bool_image : np.ndarray, block_data : str):
    """Writes a boolean image to a schematic."""
    assert bool_image.ndim == 3
    for idx, bool_value in np.ndenumerate(bool_image):
        if bool_value:
            schem.setBlock(idx, block_data)
    
def read_tiff(path : str):
    """reads a tiff stack (in ImageJ format) and returns a minecraft compatible boolean image."""
    # remove color channel
    img = tifffile.imread(path)
    if img.ndim == 4:
        img = img.sum(axis=-1)
    # rearrange axes to be minecraft (x, y, z)
    img = np.moveaxis(img, 0, 1)
    bool_img = img.astype("bool") # convert to boolean
    return bool_img

def read_npy(path : str):
    """reads a boolean image from a numpy file."""
    img = np.load(path)
    bool_img = img.astype("bool") # ensure boolean
    return bool_img

### Create schematic

In [26]:
schem = mcschematic.MCSchematic()

path = Path(output_dir) / f"{title}_organelle.tiff"
block = "minecraft:lime_concrete"
write_image(schem, read_tiff(path), block)

path = Path(output_dir) / f"{title}_organelle_membrane.tiff"
block = "minecraft:yellow_stained_glass"
write_image(schem, read_tiff(path), block)

path = Path(output_dir) / f"{title}_nucleoli.tiff"
block = "minecraft:purple_concrete"
write_image(schem, read_tiff(path), block)

path = Path(output_dir) / f"{title}_nuclei_membrane.tiff"
block = "minecraft:pink_stained_glass"
write_image(schem, read_tiff(path), block)

path = Path(output_dir) / f"{title}_membrane.tiff"
block = "minecraft:light_blue_stained_glass"
write_image(schem, read_tiff(path), block)

schem.save(output_dir, title + "_schematic", mcschematic.Version.JE_1_20)